<a href="https://colab.research.google.com/github/pedrobcavalcante/Backtest-trading-strategies/blob/main/Gerador_de_Par%C3%A2metros_para_estrat%C3%A9gias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [1]:
import backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import os.path
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
backtesting.set_bokeh_output(notebook=False)


/home/pedro/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:47: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
from backtesting.test import SMA, GOOG
# GOOG.to_csv("data/GOOG.csv")


In [3]:
def fixData(data):
    columns = ['open_time', 'Open', 'High', 'Low', 'Close', 'Volume',
               'close_time', 'qav', 'num_trades',
               'taker_base_vol', 'taker_quote_vol', 'ignore']
    df = pd.DataFrame(data, columns=columns)
    df.index = [dt.datetime.fromtimestamp(x/1000.0) for x in df.close_time]
    df.sort_values(["open_time"], ascending=False)
    df = df.astype('double')
    df['Volume'] = df['Volume'].astype('int')
    df = df.drop(['qav', 'num_trades', 'taker_base_vol',
                 'taker_quote_vol', 'ignore'], axis=1)
    return df


In [4]:
def get_bars(symbol, interval='1h'):
    root_url = 'https://api.binance.com/api/v1/klines'
    filename = "data/"+symbol+"_"+interval+".csv"
    url = root_url + '?symbol=' + symbol + '&interval=' + interval+"&limit=1001"
    print(url)
    data = json.loads(requests.get(url).text)
    df = fixData(data)
#    df.to_csv(filename, sep=';')
    return df


In [5]:
dataframe = get_bars("BTCUSDT", "1d")
dataframe


https://api.binance.com/api/v1/klines?symbol=BTCUSDT&interval=1d&limit=1001


,open_time,Open,High,Low,Close,Volume,close_time
2019-01-30 20:59:59.999,1.548806e+12,3410.04,3478.00,3387.10,3458.18,23968,1.548893e+12
2019-01-31 20:59:59.999,1.548893e+12,3457.50,3489.20,3418.80,3434.10,29607,1.548979e+12
2019-02-01 20:59:59.999,1.548979e+12,3434.10,3488.00,3401.20,3462.07,25260,1.549066e+12
2019-02-02 20:59:59.999,1.549066e+12,3462.20,3526.40,3440.29,3504.77,17920,1.549152e+12
2019-02-03 20:59:59.999,1.549152e+12,3504.06,3511.09,3426.00,3458.11,19867,1.549238e+12
...,...,...,...,...,...,...,...
2021-10-21 20:59:59.999,1.634774e+12,66001.40,66639.74,62000.00,62193.15,68538,1.634861e+12
2021-10-22 20:59:59.999,1.634861e+12,62193.15,63732.39,60000.00,60688.22,52119,1.634947e+12
2021-10-23 20:59:59.999,1.634947e+12,60688.23,61747.64,59562.15,61286.75,27626,1.635034e+12
2021-10-24 20:59:59.999,1.635034e+12,61286.75,61500.00,59510.63,60852.22,31226,1.635120e+12


In [36]:
class HiLo(Strategy):
    n1 = 1
    n2 = 1
    media = 1

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.High, self.n1)
        self.sma2 = self.I(SMA, self.data.Low, self.n2)
        self.sma3 = self.I(SMA, self.data.Close, self.media)

    def next(self):
        # if self.data.Close > self.media:
            # If sma1 crosses above sma2, close any existing
            # short trades, and buy the asset
            if self.data.Close > self.sma1:
                # self.position.close()
                self.buy()

            # Else, if sma1 crosses below sma2, close any existing
            # long trades, and sell the asset
            if self.sma2 > self.data.Close:
                self.position.close()
                # self.sell()

    def setInitialParams(self, valor1, valor2, media):
        self.n1 = valor1
        self.n2 = valor2
        self.media = media

In [46]:
def gerarParametros(dataframe, name, myStrategy, tamanho, media):
    maiorRetorno = {}
    maoirAcerto = {}
    BoasEstratégiasParaSalvar = []
    # for m in range(media):
    #     m += 1
    #     print("\nTESTE MÉDIA: ", m,end='')
    for s1 in range(tamanho):
        s1 += 1
        print("\nTESTE MSA1: ", s1,end='')
        print("sma1: ", s1, end=" ")
        for s2 in range(tamanho):
            s2 += 1
            if(s2%10 == 0 or s2 == tamanho or s2 == 1 ):
                print("\n         ", end='')
            print("sma2", s2, end=" || ")
            try:
                HiLo.setInitialParams(HiLo, s1, s2,10)
                bt = Backtest(dataframe, myStrategy,cash=100000000, exclusive_orders=True)
                output = bt.run()
                if maiorRetorno == {}:
                    maiorRetorno = {"sma1": s1, "sma2": s2, "output": output}
                if maoirAcerto == {}:
                    maoirAcerto = {"sma1": s1, "sma2": s2, "output": output}
                if output['Return [%]'] > maiorRetorno["output"]['Return [%]'] and output['# Trades'] > 9:
                    maiorRetorno = {"sma1": s1, "sma2": s2, "output": output}
                if output['Return [%]'] > 0 and output['# Trades'] > 9:
                    BoasEstratégiasParaSalvar.append(
                        {"sma1": s1, "sma2": s2, "output": output})
            except Exception as e:
                print(e)
      
    if not os.path.exists('estrategy results'):
        os.makedirs('estrategy results')
    bestParamtes = pd.DataFrame(BoasEstratégiasParaSalvar)
    bestParamtes.to_csv("estrategy results/"+name+".csv", sep=';')


In [47]:
data = json.loads(requests.get(
    "https://api.binance.com/api/v1/exchangeInfo").text)
data = data['symbols']


In [48]:
pares = []
matches = [x for x in data if x['quoteAsset'] == "USDT"]
matches.sort
for matche in matches:
    # if  not matche['symbol'] == "BTCUSDT" and not matche['symbol'] == "ETHUSDT":        
        print('\n', matche['symbol'])
        dataframe = get_bars(matche['symbol'], "1w")
        filename = matche['symbol']+"_"+"1w_HiLo_Strategy"
        gerarParametros(dataframe, filename, HiLo, 100,500)



 BTCUSDT
https://api.binance.com/api/v1/klines?symbol=BTCUSDT&interval=1w&limit=1001

TESTE MSA1:  1sma1:  1 
         sma2 1 || sma2 2 || sma2 3 || sma2 4 || sma2 5 || sma2 6 || sma2 7 || sma2 8 || sma2 9 || 
         sma2 10 || sma2 11 || sma2 12 || sma2 13 || sma2 14 || sma2 15 || sma2 16 || sma2 17 || sma2 18 || sma2 19 || 
         sma2 20 || sma2 21 || sma2 22 || sma2 23 || sma2 24 || sma2 25 || sma2 26 || sma2 27 || sma2 28 || sma2 29 || 
         sma2 30 || sma2 31 || sma2 32 || sma2 33 || sma2 34 || sma2 35 || sma2 36 || sma2 37 || sma2 38 || sma2 39 || 
         sma2 40 || sma2 41 || sma2 42 || sma2 43 || sma2 44 || sma2 45 || sma2 46 || sma2 47 || sma2 48 || sma2 49 || 
         sma2 50 || sma2 51 || sma2 52 || sma2 53 || sma2 54 || sma2 55 || sma2 56 || sma2 57 || sma2 58 || sma2 59 || 
         sma2 60 || sma2 61 || sma2 62 || sma2 63 || sma2 64 || sma2 65 || sma2 66 || sma2 67 || sma2 68 || sma2 69 || 
         sma2 70 || sma2 71 || sma2 72 || sma2 73 || sma2 74 || sma2 

KeyboardInterrupt: 

# Nova seção
